# 3. 생일 느린 순 10명 출력

In [33]:
from heap import Heap 
import os
import csv
from datetime import datetime

current_dir = os.getcwd()  
csv_path = os.path.join(current_dir, 'birthday.csv')

# 최대 힙(max heap) 인스턴스 생성
birthday_heap = Heap()

# birthday.csv 파일 읽기
try:
    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # 학번 키는 BOM 때문에 "\ufeff학번" 으로 읽히는 경우가 있음
            student_id = row.get('\ufeff학번', row.get('학번', '')).strip()
            name = row['name'].strip()
            bday_str = row['birthday'].strip()  # 생일 데이터가 "20040101"과 같이 YYYYMMDD 형식
            # 생일 항목이 빈칸이면 건너뛰기
            if bday_str == "":
                continue
            try:
                # 생일 문자열을 datetime 객체로 변환
                bday = datetime.strptime(bday_str, '%Y%m%d')
            except ValueError:
                print(f"잘못된 날짜 형식: {bday_str}")
                continue
            # 최신 생일이 우선되도록, 날짜의 ordinal 값을 우선순위 key로 사용
            key = bday.toordinal()
            # 튜플 구조: (우선순위 key, 학번, 이름, 생일)
            birthday_heap.insert((key, student_id, name, bday))
except FileNotFoundError as e:
    print("파일을 찾을 수 없습니다:", csv_path)
    raise e

# 힙에서 생일(날짜)가 늦은 순서대로 10명의 데이터를 출력 (학번 이름: 생년월일)
print("생일이 늦은 순서대로 10명의 친구:")
for _ in range(10):
    if birthday_heap.isEmpty():
        break
    key, student_id, name, bday = birthday_heap.deleteMax()
    print(f"{student_id} {name}: {bday.strftime('%Y%m%d')}")

잘못된 날짜 형식: 20241282
생일이 늦은 순서대로 10명의 친구:
******22 신수민: 20051230
******42 이서영: 20051225
******69 강민주: 20051214
******78 김민경: 20051202
******41 이서영: 20051112
******17 배시은: 20051102
******87 김여원: 20051031
******44 이서진: 20051028
******64 서홍빈: 20051024
******89 김예빈: 20051019


# 4. 같은 조원 출력

In [32]:
import csv
from circularDoublyLinkedList import CircularDoublyLinkedList

# 출력 대상 이름 리스트
target_names = [
    "박지호", "나주희", "김채현", "민고은", "김나현",
    "이서영", "안정민", "손지원", "강민주", "김민주",
    "윤혜진", "김시연", "여지혜", "두경은", "이유빈"
]

def load_birthdays(csv_filename, cdll):
    """
    birthday.csv 파일의 각 행을 읽어,
    원형 연결 리스트(cdll)에 저장합니다.
    
    CSV 파일은 첫 행에 헤더가 있고, 데이터는 dict 형태로 읽어집니다.
    """
    with open(csv_filename, mode="r", newline="", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            cdll.append(row)
    return cdll

def get_filtered_results(cdll, names):
    """
    원형 연결 리스트를 순회하면서, 각 target name에 해당하는 데이터를 
    결과 딕셔너리에 저장합니다.
    
    특히, '김민주'의 경우에는 동명이인 문제로 학번이 "******80" 인 데이터만 선택합니다.
    """
    results = {}
    dummy = cdll.getNode(-1)  # 더미 노드
    current = dummy.next       # 첫 번째 실제 노드
    while current != dummy:
        if isinstance(current.item, dict):
            name = current.item.get("name", "").strip()
            if name in names:
                if name == "김민주":
                    student_number = current.item.get("\ufeff학번", "").strip()
                    # 동명이인 처리: 학번이 "******80" 인 경우만 저장
                    if student_number == "******80":
                        results[name] = current.item
                else:
                    # 동일 이름이 여러 개인 경우, 첫 번째 데이터만 저장
                    if name not in results:
                        results[name] = current.item
        current = current.next
    return results

def print_filtered_results(results, names):
    """
    저장된 결과를 "학번 이름: 생년월일" 형식으로 출력합니다.
    """
    for name in names:
        if name in results:
            # 학번은 '\ufeff학번' 키에 저장되어 있음
            student_number = results[name].get("\ufeff학번", "").strip()
            birthday = results[name].get("birthday", "").strip()
            print(f"{student_number} {name}: {birthday}")

def main():
    cdll = CircularDoublyLinkedList()
    load_birthdays("birthday.csv", cdll)
    results = get_filtered_results(cdll, target_names)
    print_filtered_results(results, target_names)

if __name__ == "__main__":
    main()

******13 박지호: 20040728
******53 나주희: 20041104
******99 김채현: 20040409
******09 민고은: 20050214
******73 김나현: 20040203
******41 이서영: 20051112
******27 안정민: 20040501
******20 손지원: 20050620
******69 강민주: 20051214
******80 김민주: 20041026
******37 윤혜진: 20050517
******39 김시연: 20030910
******29 여지혜: 20051009
******06 두경은: 20041105
******54 이유빈: 20050601


# 5. 8장 연습문제

**01**

최대 힙에서는 더 얕은 것에 있는 원소가 더 깊은 곳에 있는 원소보다 작을 수 있다. 중요한 것은 부모-자식 관계를 유지하는 것이지, 전체 노드 간의 비교는 아니다.

예를 들면 heap[1]과 heap[2]는 깊이가 같은 노드이며, 각각의 값은 7과 10이라 하자. 값이 9인 heap[5]가 heap[2]의 자식 노드로 있으면, heap[2]의 값 10은 자식 노드 heap[5]의 값 9보다 크므로 힙의 규칙을 따른다. 이때 heap[1]은 heap[5]보다 깊이가 얕지만 값이 더 작다.

따라서 임의의 최대 힙에서 더 얕은 곳에 있는 원소가 더 깊은 곳에 있는 원소보다 더 작은 값을 가질 수 있다.

**02**

최대 힙의 마지막 원소가 항상 가장 작은 값은 아니다. 

heap[0] = 10, heap[1] = 7, heap[2] = 9이고 이 두 노드가 마지막 노드인 힙이 있다고 하자.이 힙은 heap[0] > heap[1], heap[0] > heap[2]가 되어 힙의 조건을 만족하며, heap[n-2] < heap[n-1]을 만족한다.


**03**

부모 노드의 인덱스는 (n - 2) // 2이다. 이 인덱스부터 루트 노드까지 역순으로percolateDown을 진행하여 힙 구조를 만든다. percolateDown을 수행하지 않고 건너뛰는 노드들은 리프 노드다. 리프 노드들은 부모 노드가 아니기 때문에 스며내리기가 필요하지 않다.
따라서 n - 1 - ((n - 2) // 2)로 계산하여 리프 노드의 수를 얻을 수 있습니다.

**04**

스며내리기할때 최악의 경우 시간 복잡도는 노드가 트리의 맨 아래까지 내려가야 하는 경우를 고려한다. 예를 들어, 루트 노드에서 시작하는 경우에 최대 깊이까지 내려가는 상황이 최악의 경우에 해당한다. 힙의 최대 깊이는 완전 이진 트리이므로 Θ(log n)이다.

따라서 최악의 경우 시간 복잡도는 Θ(log n)이다.

**05**

힙 구조에서는 일반적으로 삭제 작업이 루트 노드에 집중된다. 루트 노드를 삭제한 후 마지막 노드를 루트로 옮기고, 다시 힙 구조를 유지하는 작업이 필요하다다. 이 작업은 힙의 높이에 따라 O(log n)의 시간 복잡도를 가집니다.

**06**

기존의 buildHeap 방식은 각 노드마다 비교해야 하는 깊이가 다르기 때문에 총 작업 시간이 𝑂(𝑛)에 수렴한다. 위에서 아래로 스며 오르기 방식은 i번째 원소를 삽입할 때 최대 log i번 스며 오르기 해야하므로 총 작업 시간이 𝑂(nlog⁡𝑛)이다. 즉, 기존 방식에 비해 더 오래 걸려 비효율적이다.

**07**

최대 힙은 부모 노드 ≥ 자식 노드 구조이다. 임의의 원소 값이 증가한다면 원래 있던 위치의 부모 노드보다 값이 커질 수 있다. 이 경우 힙 속성이 깨지게 되므로 해당 원소를 부모 노드와 비교하며 위로 올려줘야 한다. 최악의 경우 루트까지 올라갈 수 있다. 이렇게 스며 오르기 알고리즘을 이용하면 힙의 높이에 따라 최대 O(log n) 시간이 소요되며, 최대 힙의 속성을 유지할 수 있다.

# 6. LettCode 703

In [4]:
class KthLargest(object):

    def __init__(self, k, nums):
        self.k = k
        self.heap = []
        for num in nums:
            self.add(num)

    def _sift_up(self, index):
        while index > 0:
            parent = (index - 1) // 2
            if self.heap[parent] <= self.heap[index]:
                break
            self.heap[parent], self.heap[index] = self.heap[index], self.heap[parent]
            index = parent

    def _sift_down(self, index):
        size = len(self.heap)
        while index * 2 + 1 < size: 
            smallest = index
            left = index * 2 + 1
            right = index * 2 + 2
            if left < size and self.heap[left] < self.heap[smallest]:
                smallest = left
            if right < size and self.heap[right] < self.heap[smallest]:
                smallest = right
            if smallest == index:
                break
            self.heap[index], self.heap[smallest] = self.heap[smallest], self.heap[index]
            index = smallest

    def _heappush(self, val):
        self.heap.append(val)
        self._sift_up(len(self.heap) - 1)

    def _heap_replace(self, val):
        ret = self.heap[0]
        self.heap[0] = val
        self._sift_down(0)
        return ret

    def add(self, val):
        if len(self.heap) < self.k:
            self._heappush(val)
        elif val > self.heap[0]:
            self._heap_replace(val)
        return self.heap[0]